## Code for Suppl Fig 8

In [ ]:
# using kernel .mne-python (Python 3.10.10)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pandas as pd


import mne

import h5io

from meeglet import define_frequencies


 Create adjacency matrix

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = (
    str(sample_data_folder) + "/MEG/sample/sample_audvis_filt-0-40_raw.fif"
)
raw_for_adjacency = mne.io.read_raw_fif(sample_data_raw_file)
meg_indices = raw_for_adjacency.pick_types(meg='mag')
adj_matrix = mne.channels.find_ch_adjacency(raw_for_adjacency.info, 'mag')[0]

In [ ]:
foi = define_frequencies(foi_start=1, foi_end=64, bw_oct=0.35, delta_oct=0.05)[0]

### IMPORT FEATURES

In [ ]:
features_CBU = h5io.read_hdf5('./BIOFIND/cosmeeg_CBU_2023-06-22_10-06.h5')
features_CTB1 = h5io.read_hdf5('./BIOFIND/cosmeeg_CTB_2023-06-22_10-46.h5')
features_CTB2 = h5io.read_hdf5('./BIOFIND/cosmeeg_CTB_2023-06-23_11-04.h5')

In [ ]:
features_all = features_CTB1 | features_CTB2 | features_CBU #merge 3 hdf5 files (key: patient, value : tuple cov, pow)

In [ ]:
features_all.keys()
features_all['Sub0014'][1].shape

In [ ]:
pow = np.array([features_all[subject][1] for subject in features_all]) # 0 cov, 1 pow, 2 csd
pow.shape

In [ ]:
participants_fname = './participants.tsv'
subject_df = pd.read_csv(participants_fname, delimiter='\t')
subject_df['participant_id'] = subject_df['participant_id'].str.replace('sub-', '')
subject_df = subject_df.set_index('participant_id') # creates index

### Build subject groups

In [ ]:
mean_mmse = subject_df['MMSE'].mean()
subject_df['MMSE'] = subject_df['MMSE'].fillna(mean_mmse)
mean_edu = subject_df['Edu_years'].mean()
subject_df['Edu_years'] = subject_df['Edu_years'].fillna(mean_edu)

In [ ]:
subject_df = subject_df.loc[features_all.keys()]
subject_df.head()

# Analysis PSD


In [ ]:
feature_mmse = list(subject_df['MMSE'])
feature_control = [ 1 if is_control else 0 for is_control in subject_df['group'] == 'control']
feature_converter = subject_df['Converters'].fillna(-1)

In [ ]:
psds = np.array([features_all[subject][1] for subject in features_all]) # 0 cov, 1 pow, 2 csd
psds = 10*np.log10(psds) #dB
psds_mean_chan = np.mean(psds, 1) #mean over channels
psds_mean_chan.shape

In [ ]:
del features_all, features_CBU, features_CTB1, features_CTB2

## Plot mean psd by group (control vs MCI Stable)

In [ ]:
plt.rcParams['font.size'] = 11
fig, axes = plt.subplots(1,1, figsize=[3.6,1.4],sharey=True)
psd_mean_control = np.mean(psds_mean_chan[np.array(feature_control) == 1],0)

psd_mean_nonconverter = np.mean(psds_mean_chan[np.array(feature_converter) == 0],0)
plt.plot(foi, psd_mean_control, label='Control', linewidth = 2.3)

plt.plot(foi, psd_mean_nonconverter, label='Stable MCI', linewidth = 2.3)

# Remove the legend frame
legend = plt.legend(loc='upper right', bbox_to_anchor=(0.6, 0.6))
frame = legend.get_frame()
frame.set_linewidth(0) 
plt.title('Average power spectra', fontsize=11, y=1)
plt.xlabel('Frequencies (Hz)')
plt.ylabel('Power (dB)')
plt.ylim(-280,-260)
plt.xscale('log', base =2)
plt.xticks([1,2,4,8,16,32,64],labels = [1,2,4,8,16,32,64])
#plt.xticks(plt.xticks()[0][2:-2], labels = [1,2,4,8,16,32,64])
sns.despine(offset=10, trim=True);
plt.savefig('./figures/suppl_figure8a_controlvsADprogression.pdf', bbox_inches='tight')

## Permutation F-test on sensor data averaged over all sensors

In [ ]:
# comparison for control vs MCI stable
data_condition_1 = psds_mean_chan[np.array(feature_control) == 1]
data_condition_2 = psds_mean_chan[np.array(feature_converter) == 0]
X = [data_condition_1, data_condition_2]

In [ ]:
data_condition_2.shape

## TFCE F-test method to compare averaged metrics between groups


In [ ]:
from mne.stats import permutation_cluster_test
T_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test(
    [data_condition_2, data_condition_1],
    threshold=dict(start=0, step=0.2), n_jobs=None, 
    n_permutations=10000,  
    tail=0, out_type="mask")

In [ ]:
print(cluster_p_values)

### Vizualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bootstrap
from scipy.stats import mood, norm
rng = np.random.RandomState(23)

def my_statistic(sample1, sample2, axis=0):
    statistic = np.mean(sample1) - np.mean(sample2)
    return statistic

condition1=data_condition_1
condition2=data_condition_2

results = list()

for ii in range(condition2.shape[1]):
    data = (condition2[:, ii], condition1[:, ii])
    res = bootstrap(data, my_statistic, method='basic', random_state=rng, vectorized=False)
    results.append(res)    

conf_ints = np.array([tuple(res.confidence_interval) for res in results])
conf_ints.shape
                    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bootstrap

plt.rcParams['font.size'] = 11

# mean PSD difference
mean_difference = condition2.mean(axis=0) - condition1.mean(axis=0)

# IC95 for the difference
upper_bound = conf_ints[:,1]
lower_bound = conf_ints[:,0]

fig, ax = plt.subplots(1, 1, figsize=[3.6,1.4], sharey=True)
ax.set_title('PSD difference between groups', fontsize = 11.5, y=1.05)
ax.set_xscale('log', base=2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
xticks = [1, 2, 4, 8, 16, 32, 64]
ax.set_xticks(xticks)
ax.set_xticklabels(xticks)
#ax.set_ylim(-1, 1.7)
ax.set_xlim(1, 64)
ax.axhline(0, color='black', linestyle= "--")
sns.despine(offset=5, trim=False)

# Plot the mean difference
line = ax.plot(
    foi,
    mean_difference,
    label="Mean Difference"
)

# Fill the area between upper and lower CI bounds
ax.fill_between(foi,upper_bound, lower_bound, alpha=0.5, label="95% CI")

ax.set_ylabel("PSD difference (dB)")
ax.set_xlabel("Frequencies (Hz)")

plt.savefig('./figures/figure8b_ctrvsMCIStable.pdf', bbox_inches='tight')


In [ ]:
mask = cluster_p_values <= 0.05

plt.rcParams['font.size'] = 11
fig, ax2 = plt.subplots(1, 1, figsize=[3.6, 1.4], sharey=True)
ax2.set_title('Permutation F-test on mean PSD', fontsize=11.5, y=1.05)

# Plot the main black line
hf, = ax2.plot(foi, T_obs, "black")

# Find contiguous regions of significant frequencies
inside_cluster = False
fill_plotted = False 

for i in range(len(foi)):
    if mask[i] and not inside_cluster:
        start_idx = i
        inside_cluster = True
    elif not mask[i] and inside_cluster:
        end_idx = i
        # Fill between for this cluster region
        if not fill_plotted:
            ax2.fill_between(foi[start_idx:end_idx], y1=T_obs[start_idx:end_idx], y2=0, 
                             color="green", label='Cluster P < 0.05', alpha=0.3)
            fill_plotted = True  # Set flag so that the label is added only once
        else:
            ax2.fill_between(foi[start_idx:end_idx], y1=T_obs[start_idx:end_idx], y2=0, 
                             color="green", alpha=0.3)
        inside_cluster = False

# If a cluster goes till the end of the data
if inside_cluster:
    if not fill_plotted:
        ax2.fill_between(foi[start_idx:], y1=T_obs[start_idx:], y2=0, color="green", alpha=0.3, label='Cluster P < 0.05')
    else:
        ax2.fill_between(foi[start_idx:], y1=T_obs[start_idx:], y2=0, color="green", alpha=0.3)

# Set axes labels and scales
ax2.set_xlabel("Frequencies (Hz)")
ax2.set_ylabel("F-values")
ax2.set_xscale('log', base=2)
xticks = [1, 2, 4, 8, 16, 32, 64]
ax2.set_xticks(xticks)
ax2.set_xlim(1, 64)
ax2.set_ylim(0, 10)
ax2.set_xticklabels(xticks)

ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
sns.despine(offset=5, trim=False)

ax2.legend(loc='upper right', bbox_to_anchor=(0.6, 0.6), ncol=2, fontsize=10, frameon=False)

plt.savefig('./figures/suppl_figure8c_ctrsvMCIStable.pdf', bbox_inches='tight')


In [ ]:
# print significant frequencies for clusters
print(foi[mask])